<a href="https://colab.research.google.com/github/SBhat2615/pcap-mini-project/blob/main/colab-workspace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# To show that if there is cuda tookit installed
!ls /usr/local

bin    cuda	cuda-11.8  games	       include	lib64	   man	 share
colab  cuda-11	etc	   _gcs_config_ops.so  lib	licensing  sbin  src


In [ ]:
# To show that if we have the nvcc command
!which nvcc

/usr/local/cuda/bin/nvcc


In [ ]:
# To show the property of the nvidia card(On my one, I use the K80)
!nvidia-smi

Wed Apr 19 03:37:18 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
%%writefile trial1.cu
#include <stdio.h>
#include <math.h>
#include <stdlib.h>
#define H 6
#define PIX 248
#define NUM 248 * 248
#define SIGMA 25
#ifndef max
#define max( a, b ) ( ((a) > (b)) ? (a) : (b) )
#endif
#ifndef min
#define min( a, b ) ( ((a) < (b)) ? (a) : (b) )
#endif
void dataread(float *y);
int distance(int i, int j, int k, int l);
void lambda(float *y, float *lold, float *lnew, float *p);
void renew(float *lold,float *lnew);
void spread(float *p);
void datasave(float *l);
int main(void){
int i;
float *y, *lold, *lnew, *p, ttt;
y = malloc(NUM * sizeof(float));
lold = malloc(NUM * sizeof(float));
lnew = malloc(NUM * sizeof(float));
p = malloc((2 * (H + 1) * (H + 1)) * sizeof(float));
spread(p);
dataread(y);
dataread(lold);
for(i = 0; i < 40; i++){
lambda(y, lold, lnew, p);
renew(lold, lnew);
printf("%d \n",i);
}
datasave(lnew);
free(p);
free(y);
free(lnew);
free(lold);
return 0;
}
/*READING THE DATA*/
void dataread(float *y){
int i;
FILE *fp;
fp = fopen("mylenna","r");
for(i = 0; i < NUM; i++){
fscanf(fp,"%f ",&y[i]);
}
return;
}
/*DEFINING A NORM*/
int distance(int i, int j, int k, int l){
int d;
d = (i - k) * (i - k) + (j - l) * (j - l);
return d;
}
/*LAMBDA VECTOR*/
void lambda(float *y, float *lold, float *lnew, float *p){
int i1, i2, j1, j2, k1, k2;
float temp, tmp;
for(i1 = 0; i1 < PIX; i1++){
for(i2 = 0; i2 < PIX; i2++){
tmp = 0;
for(j1 = max(0, i1 - H + 1); j1 < min(i1 + H - 1, PIX); j1++){
for(j2 = max(0, i2 - H + 1); j2 < min(i2 + H - 1, PIX); j2++){
temp = 0;
for(k1 = max(0, i1 - H + 1); k1 < min(i1 + H - 1, PIX); k1++){
for(k2 = max(0, i2 - H + 1); k2 < min(i2 + H - 1, PIX); k2++){
temp = temp + lold[k1 * PIX + k2] * p[distance(j1,j2,k1,k2)];
}
tmp = tmp + y[j1 * PIX + j2] * (p[distance(j1,j2,i1,i2)])/temp;
}
}
lnew[i1 * PIX + i2] = lold[i1 * PIX + i2] * tmp;
}
}
return;
}
/*Point Spread Function*/
void spread(float *p){
int i, j;
float temp = 0;
for(i = -H; i <= H; i++){
for(j = -H; j <= H; j++){
p[i * i + j * j] = exp(-((float)(i * i + j * j))/SIGMA);
temp += p[i * i + j * j];
}
}
for(i = -H; i <= H; i++)
for(j = -H; j <= H; j++)
p[i * i + j * j] = p[i * i + j * j]/temp;
return;
}
/*RENEW THE VALUE OF LAMBDA VECTOR*/
void renew(float *lold,float *lnew){
int i;
for(i = 0; i < NUM; i++)
lold[i] = lnew[i];
return;
}
/*SAVING THE DATA*/
void datasave(float *l){
int i, j;
FILE *fp;
fp = fopen("mylenn","w");
for(i = 0; i < PIX; i++){
for(j = 0; j < PIX; j++){
fprintf(fp,"%f ",l[i * PIX + j]);
}
fprintf(fp,"\n");
}
return;
}

Overwriting l9q1a.cu


In [ ]:
!nvcc -arch=sm_37 -gencode=arch=compute_37,code=sm_37 l9q1a.cu -o l9q1a

nvcc warning : The 'compute_35', 'compute_37', 'sm_35', and 'sm_37' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).


In [ ]:
!./l9q1a

Enter the value of m: 2
Enter the value of n: 2
Enter input matrix A:
2 4 5 6
Enter input matrix B:
7 8 3 2
cuda successfully ran!
cuda successfully ran!
Result matrix is:
9	12	
8	8	
